In [1]:
from utility_programs.read_routines import SAMI

In [2]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator, griddata
import numpy as np


from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr

import matplotlib.pyplot as plt

import cartopy.crs as ccrs

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
import interpolate_outputs

In [4]:
samipath = '/petastore/phil/Prasoon_satellite_project/more_runs/20130317/sami-gitm-coupled/'

In [57]:
interpolate_outputs.do_interpolations(samipath,
                                    out_path='/petastore/phil/Prasoon_satellite_project/satfiles/',
                                      dtime_sim_start='20130316',
                                    cols='edens',
                                      engine=None,
                                    save_delauney=True,
                                    show_progress=True)

attempting to reuse existing triangulation file


Reading in SAMI data:   0%|          | 0/596 [00:00<?, ?it/s]

  0%|          | 0/597 [00:00<?, ?it/s]

## If you make any changes to the source code, reimport it with:

(Then you don't have to restart the kernel)

In [7]:
import importlib

In [56]:
interpolate_outputs = importlib.reload(interpolate_outputs)

## Read in satellite file:

In [58]:
import pandas as pd

In [59]:
allsatdf = pd.read_csv('/petastore/phil/Prasoon_satellite_project/satfiles/20130317/DMSP_all-with_datetime.csv')

### Now grab just a single satellite's data

In [68]:
firstsat = allsatdf.loc[allsatdf.sat == 16].reset_index(drop=True)
# make sure to reset index & drop bad values, otherwise it's harder
firstsat

,glat,glon,time,sat,dayofyear,dtime
0,2.8,261.2,0.0,16,75,2013-03-16 00:00:00
1,6.3,260.5,60.0,16,75,2013-03-16 00:01:00
2,9.8,259.7,120.0,16,75,2013-03-16 00:02:00
3,13.3,258.9,180.0,16,75,2013-03-16 00:03:00
4,16.8,258.0,240.0,16,75,2013-03-16 00:04:00
...,...,...,...,...,...,...
4315,56.7,96.1,86100.0,16,77,2013-03-18 23:55:00
4316,53.3,94.3,86160.0,16,77,2013-03-18 23:56:00
4317,49.9,92.7,86220.0,16,77,2013-03-18 23:57:00
4318,46.5,91.2,86280.0,16,77,2013-03-18 23:58:00


In [76]:
latlon = [firstsat.glat, firstsat.glon]
latlon = np.array(latlon)

In [80]:
# no alt in files, just assume it's 840 km. Add that to latlon object
alt = [840 for i in range(len(latlon[0]))]

In [81]:
# add in alt.
# Theres's a better way to do this but I don't remember...
latlonalt = np.array([latlon[0], latlon[1], alt])

In [84]:
latlonalt

array([[  2.8,   6.3,   9.8, ...,  49.9,  46.5,  43. ],
       [261.2, 260.5, 259.7, ...,  92.7,  91.2,  89.9],
       [840. , 840. , 840. , ..., 840. , 840. , 840. ]])

### Now send this in to the interpolation function

> I'm going to add a catch to make sure the file is readable by everyione in the `phil` group

Also make sure we get the dataset back. But this isn't necessary! Going to show an example where we just recalculate things and then read it back.

In [95]:
out_path = '/petastore/phil/Prasoon_satellite_project/'

sami_data_path = '/petastore/phil/Prasoon_satellite_project/more_runs/20130317/sami-gitm-coupled/'

In [ ]:
ds = interpolate_outputs.do_interpolations(sami_data_path=sami_data_path,
                                     dtime_sim_start='20110316',
                                     out_latlonalt=latlonalt,
                                     out_path=out_path,
                                     out_runname='-at-sat_16',
                                     save_delauney=True,
                                     max_alt=1500, #to make running faster. Not necessary.
                                     cols='edens',
                                    show_progress=True,
                                    engine='h5netcdf', #change to None if this makes errors
                                      return_to_ds_too=True
                                     )

In [98]:
## Don't forget to change the permissions!!

!chmod -R 770 {out_path}


#### To not get the datset back and then reafd it from the created file later:

i.e. you can run interpolation code in a console and then start a jupyter notebook to analyze things

In [ ]:
interpolate_outputs.do_interpolations(sami_data_path=sami_data_path,
                                     dtime_sim_start='20110316',
                                     out_latlonalt=latlonalt,
                                     out_path=out_path,
                                     out_runname='-at-sat_16',
                                     save_delauney=True,
                                     max_alt=1500, #to make running faster. Not necessary.
                                     cols='edens',
                                    show_progress=True,
                                    engine='h5netcdf', #change to None if this makes errors
                                     )

In [ ]:
## Don't forget to change the permissions!!

!chmod -R 770 {out_path}

In [ ]:
# then read the created file with:

import xarray as xr

ds = xr.open_datset(os.path.join(out_path, 'SAMI_REGRID-at-sat-16.nc'),
                   engine='h5netcdf') #set engine to the same one used to calculate interpolation.


## That's about all I have time for now. 


Should be simple enough, lol. I'm sure there will be problems. Let me know what they are and I'll fix them.

You can loop through multiple satellites by changing the '16' in each to the values of each satellite listed in the satfile. 

In [102]:
# see the available satellites:

np.unique(allsatdf.sat)

array([16, 17, 18])

In [ ]:
# loop through them:

for satnum in np.unique(allsatdf.sat):
    
    #set up output grid
    satnow = allsatdf.loc[allsatdf.sat == satnum].reset_index(drop=True)
    latlon = [firstsat.glat, firstsat.glon]
    latlon = np.array(latlon)
    alt = [840 for i in range(len(latlon[0]))]
    latlonalt = np.array([latlon[0], latlon[1], alt])
    
    interpolate_outputs.do_interpolations(sami_data_path=sami_data_path,
                                     dtime_sim_start='20110316',
                                     out_latlonalt=latlonalt,
                                     out_path=out_path,
                                     out_runname='-at-sat_' + str(satnum), 
                                          # to change the output name for each sat
                                     save_delauney=True,
                                     max_alt=1500, #to make running faster. Not necessary.
                                     cols='edens',
                                    show_progress=True,
                                    engine='h5netcdf', #change to None if this makes errors
                                     )


Then you have to read each satellite file individually with the code shown above